In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util
from preprocessor import Processor

In [3]:
util = Util()
cleaner = DataCleaner()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
news_df=util.read_from_dvc("data/news.csv",repo,"news-v0",low_memory=False)
news_df=news_df.sample(frac=1)

# cleaned_df=util.read_from_file('../data/news.csv')

In [5]:
train_news=news_df.head(8)
test_news=news_df.tail(2)

In [6]:
full_processed_df=processor.prepare_text(train_news)
description_processed=processor.prepare_text(train_news,columns=["Description"])
title_processed=processor.prepare_text(train_news,columns=["Title"])
body_processed=processor.prepare_text(train_news,columns=["Body"])
unprocessed=train_news

In [7]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)

In [8]:
processor.prepare_tuner(full_processed_df)

tuner prepared successfuly
